## <CENTER> Recency, Frequency, Monetary (RFM) segmentation

#  <center>  <img src="https://d35fo82fjcw0y8.cloudfront.net/2018/03/01013508/Incontent_image.png" width="600"> <center> 

#### ONLINE RETAIL DATASET:
Dataset used for this analysis: https://archive.ics.uci.edu/ml/datasets/online+retail <p>

<b>Recency:</b> Measure how recent was the customer last purchase.<br>
<b>Frequency:</b> Measure how many purchases does the customer has done (last 12 months). <br>
<b>Monetary Value:</b> Measure how much does the customer spend (last 12 months).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Load data & Preprocess

In [3]:
retail=pd.read_excel('Online Retail.xlsx')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
retail.shape

(541909, 8)

In [5]:
retail.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

### Initial verifications

In [6]:
##Check nulls
np.sum(retail.isna())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

The analysis will be about the customers so we drop the unidentified ones

In [7]:
##Drop nulls
retail.dropna(subset=['CustomerID'],inplace=True)
np.sum(retail.isna())

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
#identify duplicated rows
sum(retail.duplicated(subset=None,keep=False)) ##Sumar los True para verificar si hay un duplicado

10062

In [9]:
##See duplicated rows
retail[retail.duplicated(subset=None, keep=False)==True].sort_values(by=['InvoiceNo','StockCode']).head(6)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom


In [11]:
#drop duplicated rows
retail.drop_duplicates(subset=None,keep='first',inplace=True)
##verify
sum(retail.duplicated(subset=None,keep=False)) 

0

In [29]:
###Top 5 countries - by quantity
summary=retail.groupby("Country").size().reset_index()\
.rename(columns={0:"Quantity"}).set_index('Country')
summary['%Rep']=round(summary["Quantity"]/retail.shape[0],4)*100
##sorting 
summary.sort_values(by='Quantity', ascending=False).head()

,Quantity,%Rep
Country,,
United Kingdom,356728,88.83
Germany,9480,2.36
France,8475,2.11
EIRE,7475,1.86
Spain,2528,0.63


For this example the cancelled tickets are going to be removed

In [31]:
##quantity of cancelled tickets
retail[retail["Quantity"] < 0].shape

(8872, 8)

In [32]:
## Keep the noncancelled tickets
retail_clean = retail[retail["Quantity"] > 0]
retail_clean.shape

(392732, 8)

In [37]:
## Add Total price 
def total_price(x): return x.Quantity*x.UnitPrice
###
retail_clean['Total_price']=retail_clean.apply(total_price, axis=1)
retail_clean.head()

D:\Program_files\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## Build RFM Matrix for the Dataset

<b>Recency:</b> days since last customer transaction.<br>
<b>Frequency:</b> number of transactions (last 12 months). Could be also the average frequency per month but depends on the analysis.<br>
<b>Monetary Value:</b> number of transactions (last 12 months).

Note: 12 months is a standard but could be any other quantity of months

### Recency

In [38]:
from datetime import date
import time
import datetime

In [39]:
##Data - first day 
retail_clean['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [40]:
##Data - last day 
retail_clean['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [44]:
##Calculate quantity of days since last purchase of a customer
def cal_recency(val):
    last_day=retail_clean['InvoiceDate'].max()
    return (last_day-val).days

In [45]:
%%time
retail_clean['Recency']=retail_clean['InvoiceDate'].apply(cal_recency)
retail_clean.head()

Wall time: 7min 15s


D:\Program_files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_price,Recency
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,373
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,373
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373


In [47]:
##Change datatype of customerID
retail_clean=retail_clean.astype({'CustomerID': 'int'})

In [67]:
##Build DF : Customer -  min Recency
Recency=retail_clean.groupby('CustomerID').agg({'Recency':'min'}).reset_index()
Recency=Recency.astype({'Recency': 'int'})
print(Recency.shape)
Recency.head()

(4339, 2)


,CustomerID,Recency
0,12346,325
1,12347,1
2,12348,74
3,12349,18
4,12350,309


In [64]:
customers.dtypes

CustomerID    int64
Recency       int32
dtype: object

### Frequency

In [66]:
##Sum of unique purchase transactions (count unique invoiceNo per customer)
Freq=retail_clean[['CustomerID','InvoiceNo']]\
.drop_duplicates(subset=None,keep='first')\
.groupby('CustomerID').agg({'InvoiceNo':'count'})\
.reset_index().rename(columns={'InvoiceNo':"Frequency"})
##Frequency DF shape
print(Freq.shape)
Freq.head()

(4339, 2)


,CustomerID,Frequency
0,12346,1
1,12347,7
2,12348,4
3,12349,1
4,12350,1


### Monetary Value

In [70]:
##Total spend by each customer
Monto=retail_clean[['CustomerID','Total_price']]\
.groupby('CustomerID').agg({'Total_price':'sum'})\
.reset_index().rename(columns={'Total_price':"M_Value"})
##DF - Shape
print(Monto.shape)
Monto.head()

(4339, 2)


,CustomerID,M_Value
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


## Customer Dataframe with RFM rates

In [75]:
##Merge DF Recency - DF Frequency - DF Monetary Val
RFM=pd.merge(pd.merge(Recency,Freq, 'left'),Monto, 'left')
RFM.head()

,CustomerID,Recency,Frequency,M_Value
0,12346,325,1,77183.60
1,12347,1,7,4310.00
2,12348,74,4,1797.24
3,12349,18,1,1757.55
4,12350,309,1,334.40


In [77]:
##Check shape
RFM.shape, Recency.shape, Freq.shape, Monto.shape

((4339, 4), (4339, 2), (4339, 2), (4339, 2))

## RFM Analysis

RFM values can be grouped in several ways:

- Percentiles (quantiles)</b> <br>
- Pareto 80/20 cut:</b> <br>
- Custom - based on business knowledge</b>

### Calculating percentiles
Steps:

1) Sort customers based on an specific metric <br>
2) Break customers into a pre-defined number of groups of equal size <br>
3) Assign a label to each group

#### Percentiles with Spend Value (Monetory Value)

In [79]:
RFM[['CustomerID','M_Value']].head(2)

,CustomerID,M_Value
0,12346,77183.6
1,12347,4310.0


TIP : Assign the labels in such a way that the greater one represents greater importance

In [84]:
##Example for Quartile
spend_quartiles = pd.qcut(RFM['M_Value'], q=4, labels=range(1,5))
RFM['Spend_Quartile'] = spend_quartiles
RFM.sort_values('M_Value').head()

,CustomerID,Recency,Frequency,M_Value,Spend_Quartile
685,13256,13,1,0.00,1
3218,16738,297,1,3.75,1
1794,14792,63,1,6.20,1
3015,16454,44,2,6.90,1
4099,17956,248,1,12.75,1


For the column "Spend_Quartile" : <br>
Here the value of 1 represents the type of customer that spend less and the value of 4 represents the opposite

#### Percentiles with Recency 

In [88]:
# Create numbered labels (reversed)
r_labels = list(range(4, 0, -1))
# Divide into groups based on quartiles
recency_quartiles = pd.qcut(RFM['Recency'], q=4, labels=r_labels)
# Create new column
RFM['Recency_Quartile'] = recency_quartiles
# Sort recency values from lowest to highest
RFM.sort_values('Recency').head()

,CustomerID,Recency,Frequency,M_Value,Spend_Quartile,Recency_Quartile
1751,14730,0,8,1897.57,4,4
1731,14702,0,17,3251.79,4,4
137,12518,0,5,2056.89,4,4
3513,17144,0,2,984.57,3,4
3695,17405,0,3,1031.41,3,4


"THE LOWER THE RECENCY, THE HIGHER THE QUARTILE VALUE"

For the column "Recency_Quartile" : <br>
Here the value of 1 represents the type of customer that have high recency and the value of 4 represents the opposite

#### Custom labels - Recency

In [91]:
##Create name labels as string in a descending order (in case of Recency)
r_labels = ['Active', 'Lapsed', 'Inactive', 'Churned']
# Divide into groups based on quartiles 
recency_quartiles = pd.qcut(RFM['Recency'], q=4, labels=r_labels)

# Create new column
RFM['Recency_Quartile'] = recency_quartiles 

# Sort values from lowest to highest
RFM.sort_values('Recency').head()

,CustomerID,Recency,Frequency,M_Value,Spend_Quartile,Recency_Quartile
1751,14730,0,8,1897.57,4,Active
1731,14702,0,17,3251.79,4,Active
137,12518,0,5,2056.89,4,Active
3513,17144,0,2,984.57,3,Active
3695,17405,0,3,1031.41,3,Active


### Group by to see Quantity of customers assigned to each label

In [92]:
RFM.groupby('Recency_Quartile').size()

Recency_Quartile
Active      1126
Lapsed      1063
Inactive    1066
Churned     1084
dtype: int64

In [93]:
RFM.groupby('Spend_Quartile').size()

Spend_Quartile
1    1085
2    1085
3    1084
4    1085
dtype: int64

# Calculate RFM metrics

Another way to create the DataFrame for RFM metrics.
This one is more efficient.

In [102]:
##Dataset
retail_clean.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_price,Recency
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,373
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,373


In [104]:
##Print min and max dates of the dataset
print('Min:{}; Max:{}'.format(min(retail.InvoiceDate),
                              max(retail.InvoiceDate)))

Min:2010-12-01 08:26:00; Max:2011-12-09 12:50:00


In [103]:
##Create and hypothetical snapshot_date as if we're doing analysis recently
snapshot_date = max(retail.InvoiceDate) + datetime.timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [122]:
# Aggregate data on a customer level
datamart = retail_clean.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    #'InvoiceNo': pd.Series.nunique,
    'InvoiceNo': 'count',
    'Total_price': 'sum'})
# Rename columns for easier interpretation
datamart.rename(columns = {'InvoiceDate': 'Recency',
                           'InvoiceNo': 'Frequency',
                           'Total_price': 'MonetaryValue'}, inplace=True)
# Check the first rows
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346,326,1,77183.60
12347,2,182,4310.00
12348,75,31,1797.24
12349,19,73,1757.55
12350,310,17,334.40


NOTE: When assign labels always be careful. Remember that the lower value means less importance and the higher the value means higher importance

### Recency quartile

In [127]:
r_labels = range(4, 0, -1)
r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)
##Column R
datamart = datamart.assign(R = r_quartiles.values)

### Frequency and Monetary quartiles

In [128]:
f_labels = range(1,5)
m_labels = range(1,5)
f_quartiles = pd.qcut(datamart['Frequency'],q=4
                      , labels = f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], 4, labels = m_labels)
##Column F
datamart = datamart.assign(F = f_quartiles.values)
##Column M
datamart = datamart.assign(M = m_quartiles.values)

In [129]:
datamart.head()

,Recency,Frequency,MonetaryValue,F,M,R
CustomerID,,,,,,
12346,326,1,77183.60,1,4,1
12347,2,182,4310.00,4,4,4
12348,75,31,1797.24,2,4,2
12349,19,73,1757.55,3,4,3
12350,310,17,334.40,1,2,1


### Build RFM Segment and RFM Score

<b>RFM Segment:</b> Concatenate string of the RFM Values.<br>
<b>RFM Score:</b> Sum of RFM Values. Indicates a relative customer value.<br>

In [130]:
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

In [132]:
datamart.head(10)

,Recency,Frequency,MonetaryValue,F,M,R,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346,326,1,77183.60,1,4,1,114,6.0
12347,2,182,4310.00,4,4,4,444,12.0
12348,75,31,1797.24,2,4,2,224,8.0
12349,19,73,1757.55,3,4,3,334,10.0
12350,310,17,334.40,1,2,1,112,4.0
12352,36,85,2506.04,3,4,3,334,10.0
12353,204,4,89.00,1,1,1,111,3.0
12354,232,58,1079.40,3,3,1,133,7.0
12355,214,13,459.40,1,2,1,112,4.0


# Analyzing RFM table

## Largest RFM segments
Investigate the size of the segments 

In [133]:
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    450
111    381
344    217
122    206
211    179
233    168
333    164
222    157
433    149
322    133
dtype: int64

## Filtering on RFM segments
Use filters by the column RFM segments

In [134]:
##Observe the bottom RFM Segment
datamart[datamart['RFM_Segment']=='111'][:5]

,Recency,Frequency,MonetaryValue,F,M,R,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12353,204,4,89.00,1,1,1,111,3.0
12361,287,10,189.90,1,1,1,111,3.0
12401,303,5,84.30,1,1,1,111,3.0
12402,323,11,225.60,1,1,1,111,3.0
12441,367,11,173.55,1,1,1,111,3.0


## Summary metrics per RFM Score
Analize metrics for each group

In [135]:
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count'] }).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3.0         260.7       8.2         157.4   381
4.0         177.2      13.6         240.0   388
5.0         152.9      21.2         366.6   518
6.0          95.9      27.8         820.1   457
7.0          79.4      37.9         757.1   464
8.0          64.1      56.0         987.3   454
9.0          45.9      78.7        1795.1   414
10.0         32.4     110.5        2056.4   426
11.0         21.3     186.9        4062.0   387
12.0          7.2     367.9        9285.9   450

## Grouping into named segments
Use RFM score to group customers into Gold, Silver and Bronze segments.

In [139]:
##Define a function and define subsegments using RFM score
def segment_me(df):
    if df['RFM_Score'] >= 9:
        return '1. Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return '2. Silver'
    else:
        return '3. Bronze'

In [140]:
datamart['General_Segment'] = datamart.apply(segment_me, axis=1)

In [141]:
datamart.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
1. Gold            26.4     189.4        4394.7  1677
2. Silver          99.8      35.3         720.7  1893
3. Bronze         218.6      10.9         199.1   769